<a href="https://colab.research.google.com/github/MatteoRigoni/MachineLearningPlayground/blob/master/MachineLearning_TecnicheAvanzate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tuning degli iperparamteri

In [ ]:
import numpy as np
np.random.seed(1)
import pandas as pd

import scipy

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import make_pipeline

from sklearn.model_selection import cross_val_score, learning_curve, validation_curve

import matplotlib.pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # per printare su una stessa cella

In [ ]:
df = pd.read_csv(
    'https://archive.ics.uci.edu/ml/'
    'machine-learning-databases'
    '/breast-cancer-wisconsin/wdbc.data',
    header=None
)
df.head()
df.describe()

In [ ]:
from sklearn.preprocessing import LabelEncoder

X, y = df.iloc[:, 2:].values, df.iloc[:, 1].values
len(X), len(y)

le = LabelEncoder()
y = le.fit_transform(y)
le.classes_ #binario
len(y)

In [ ]:
#split del dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=.2, stratify=y, random_state=1
)
len(y_train), len(y_test)

In [ ]:
#creazione pipeline di linear regression
pipe_lr = make_pipeline(
    StandardScaler(), #scala prima tutti gli input
    PCA(n_components=2),
    LogisticRegression()
)
pipe_lr.fit(X_train, y_train)
y_pred = pipe_lr.predict(X_test)
test_acc = pipe_lr.score(X_test, y_test)
print(f'Test accuracy: {test_acc:.3f}')

###Cross Validation

Nel processo di tuning degli iperparametri del modello, è cruciale non usare sempre lo stesso dataset di test, perchè questo porterebbe a trovare una soluzione ottima con alto bias. La corretta procedura, invece, deve essere quella di usare un dataset di validation estratto dal dataset di training. Una possibilità sarebbe quella di usare la tecnica del holdout, in cui il dataset totale viene suddiviso in 3 sotto insiemi: [train, validation, test], in genere con percentuali del tipo [70%, 10%, 20%], rispettivamente. Tuttavia, questa tecnica dipende fortemente dal unico modo utilizzato per splittare il dataset.

Una pratica ancora più performante è la tecnica dello Stratified Cross-Validation. In tale tecnica, il dataset iniziale viene suddivo in train e test, successivamente (in fase di evaluation) il dataset di train viene suddiviso in K folds, per cui K-1 vengono usati come dataset di training e il K-esimo come validation. La procedura viene ripetuta un numero K di volte, in cui il K-esimo dataset di validation è sempre differente dal caso precedente. Lo score finale sarà ottenuto prendendo la media delle K iterazioni. Inoltre, il metodo assicura che, in ogni iterazione il numero delle classi rimane constante (stratified), in modo da trainare i K modelli, in situazioni simili, per quanto riguarda il numero di labels.

A fine della procedura, viene trainato un unico modello, usando le configurazioni degli iperparametri ottimi, ed utilizzando l'intero dataset di training (train + validation == 80% dei dati) e usando il dataset di test (mai utilizzato finora) per valutare l'errore di generalizzazione.



In [ ]:
from sklearn.model_selection import StratifiedKFold

KFold = StratifiedKFold(n_splits=10).split(X_train, y_train)

scores = []
for k, (train, test) in enumerate(KFold):
    _ = pipe_lr.fit(X_train[train], y_train[train])
    scores.append(pipe_lr.score(X_train[test], y_train[test]))
    print(
        f'Fold {k+1:02} '
        f'Class distr. : {np.bincount(y_train[train])} '
        f'CV score: {scores[-1]:.3f}')


print(f'CV accuracy averaged: {np.mean(scores):.3f} +/- {np.std(scores):.3f}')
# the same thing, but less verbose ...
scores = cross_val_score(
    estimator=pipe_lr,
    X=X_train,
    y=y_train,
    cv=10, # 10-folds
    n_jobs=-1 # exploits all available cpus
)
print(f'CV accuracy scores:\n{scores}\n')
print(f'CV accuracy: {np.mean(scores):.3f} +/- {np.std(scores):.3f}')

###Validation curve

La validation curve visualizza l'accuratezza, sul dataset di training e di validation, al variare dei valori di alcuni iperparametri del modello. Il suo plot aiuta ad individuare, in modo grafico, il range di valori entro i quali, gli iperparametri danno un accuratezza sul validation test ottima.


In [ ]:
def plot_validation_curve(estimator, X, y, param_range, param_name, cv=10):
    """Validation curve plot: shows accuracy values for different values of a select parameters.

    --Params
     - estimator: a scikit-learn estimator,
     - X, y: numpy ndarrays
     - param_range: List, the range values the paramete must explor
     - param_name: str, the name of the parameter
     -cv: int, the number of folds for the stratified cross validation
     """

    train_score, test_score = validation_curve(
        estimator=estimator,
        X=X,
        y=y,
        param_range=param_range,
        param_name=param_name,
        cv=cv
    )

    train_avg, train_std = np.mean(train_score, axis=1), np.std(train_score, axis=1)
    test_avg, test_std = np.mean(test_score, axis=1), np.std(test_score, axis=1)

    plt.plot(param_range, train_avg, color='blue', marker='o', markersize=5, label='Training Accuracy')
    plt.fill_between(param_range, train_avg + train_std, train_avg - train_std, alpha=.15, color='blue')
    plt.plot(param_range, test_avg, color='green', marker='s', markersize=5, linestyle='--', label='Validation Accuracy')
    plt.fill_between(param_range, test_avg + test_std, test_avg - test_std, alpha=.15, color='green')
    plt.legend()
    plt.grid()
    plt.xlabel(f"Parameter {param_name.split('__')[-1]}")
    plt.ylabel('Accuracy')
    plt.ylim([0.8, 1.0])
    plt.xscale('log')
    plt.show()

pipe_lr = make_pipeline(
    StandardScaler(),
    LogisticRegression(penalty='l2', max_iter=10000)
)
plot_validation_curve(pipe_lr, X_train, y_train, [0.001, 0.01, 0.1, 1.0, 10.0, 100.0], 'logisticregression__C')

In [ ]:
#grid search, per scelta degli iperparametri che sono impostati prima dell'addestramento
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

pipe_svc = make_pipeline(
    StandardScaler(),
    SVC(random_state=1)
)

param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
param_grid = [
    {'svc__C': param_range,
    'svc__kernel': ['linear']},

    {'svc__C': param_range,
    'svc__gamma': param_range,
    'svc__kernel': ['rbf']}]

grid = GridSearchCV(
    estimator=pipe_svc,
    param_grid=param_grid,
    scoring='accuracy',
    cv=10,
    refit=True,
    n_jobs=-1
)

gs = grid.fit(X_train, y_train)
gs.best_score_
gs.best_params_

In [ ]:
clf = gs.best_estimator_
_ = clf.fit(X_train, y_train)
print(f'Test accuracy: {clf.score(X_test, y_test):.3f}')

In [ ]:
#random search ,utile per molti parametri, grid trova combinazione migliore ma molto oneroso elegato a range fissi
param_range = scipy.stats.loguniform(0.0001, 1000.0) #prendiamo range con probabilità su distribuzione uniforme, quindi senza un bias prefissato
param_range.rvs(10)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = [
    {"svc__C": param_range,
    "svc__kernel": ["linear"]},
    {"svc__C": param_range,
    "svc__gamma": param_range,
    "svc__kernel": ["rbf"]}
]

rs = RandomizedSearchCV(
    estimator=pipe_svc,
    param_distributions=param_grid, # note the *ditributions part
    scoring='accuracy',
    refit=True,
    n_iter=20,
    cv=10,
    random_state=1,
    n_jobs=-1
)

_ = rs.fit(X_train, y_train)
rs.best_score_
rs.best_params_

#ha permesso di esplorare dei valori che con il grid search non avremmo esplorato

In [ ]:
#estraiamo anche qui il  best estimator per fare il fit
clf = rs.best_estimator_
_ = clf.fit(X_train, y_train)
print(f'Test accuracy: {clf.score(X_test, y_test):.3f}')

In [ ]:
#infine proviamo halving random search,  che è come il random, ma non usa tutto il dataset in ogni iterazione, permettendo di velocizzare il processo
# è una features sperimentale: bisogna importare questa macro, perchè l'API potrebbe cambiare senza deprecation cycle
from sklearn.experimental import enable_halving_search_cv
# adesso importa il metodo
from sklearn.model_selection import HalvingRandomSearchCV

hs = HalvingRandomSearchCV(
    estimator=pipe_svc,
    param_distributions=param_grid,
    n_candidates='exhaust', # scegli un numero di candidati tale da sfruttare tutto il dataset
    resource='n_samples',
    factor=1.5, # quanti candidati saranno scartati ad ogni iterazione (100%/1.5=66% rimarranno)
    random_state=1,
    n_jobs=-1
    )

_ = hs.fit(X_train, y_train)
hs.best_score_
hs.best_params_


In [ ]:
clf = hs.best_estimator_
_ = clf.fit(X_train, y_train)
print(f'Test accuracy: {clf.score(X_test, y_test):.3f}')

#Pipeline

In [ ]:
# data-preprocessing
from sklearn.impute import SimpleImputer # imputation transformer for completing nan values
from sklearn.preprocessing import StandardScaler, OneHotEncoder
# feature selector
from sklearn.decomposition import PCA
# model
from sklearn.linear_model import LinearRegression
# composer and pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# load dataset
from sklearn.datasets import fetch_openml

In [ ]:
X, y = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)
X.dtypes

In [ ]:
numerical_features = [col for col in X.columns if X[col].dtypes == 'float64']
cat_features = [col for col in X.columns if X[col].dtypes == 'category']
len(numerical_features), len(cat_features)

In [ ]:
numerical_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='constant')),
        ('one-hot', OneHotEncoder(handle_unknown='ignore'))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, cat_features)
        ]
)

my_pipe = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('pca', PCA()),
        ('clf', LinearRegression())
    ]
)

my_pipe

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=.2, random_state=1
)

In [ ]:
my_pipe.fit(X_train, y_train)
my_pipe.predict(X_train)
y_pred= my_pipe.predict(X_test)

In [ ]:
import joblib

#esportazione pipe
joblib.dump(my_pipe, 'my_pipe.joblib')

my_pipe = joblib.load('my_pipe.joblib')
my_pipe.predict(X_train)

#Serie storiche

In [ ]:
!git clone https://github.com/ProfAI/machine-learning-avanzato.git

In [ ]:
#utils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

#packages
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa.api as tsa
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # per printare su una stessa cella


In [ ]:
df = pd.read_csv("machine-learning-avanzato/datasets/coin_Bitcoin.csv")
df.head()
len(df)
df.Date.min(), df.Date.max()

In [ ]:
#preprocessing
df['Date'] = df.Date.apply(pd.to_datetime)
df = df.loc[:,['Date','High','Low','Open','Close']]
df= df.set_index('Date', drop=True)
df.head()

In [ ]:
df.Close.plot(title="BitCoin Closing Price")
#si evince che è non stazionaria, dimostriamolo

from statsmodels.tsa.stattools import adfuller
def test_ADF(x,  pvalue=0.05):
  result = adfuller(x)
  if result[1] <= pvalue:
    return True
  else:
    return False

test_ADF(df.Close)

#dobbiamo riportarlo a una serie stazionaria, visto che non  lo è, dato il p-value  alto, non possiamo escludere che non sia una serie non stazionaria
#prendiamo i valori logaritmi...
df['Close_log'] = np.log(df.Close)
df['Close_log'].plot()
test_ADF(df['Close_log'])
#...non basta, non è ancora stazionaria
df['Close_log_diff'] = df['Close_log'] - df['Close_log'].shift(1)
df.dropna(inplace=True)
_ = df['Close_log_diff'].plot()
test_ADF(df['Close_log_diff'])
#...ora risulta stazionaria, quindi con parametro 1

#ora cerchiamo che valore deve avere per averlo autoregressivo
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
fix, ax = plt.subplots(1, sharex=True, figsize=(8,8))
plot_acf(df['Close_log_diff'], lags=40, ax=ax)

from statsmodels.tsa.arima.model import ARIMA
model = ARIMA(df.Close, order=(1,1,0))
model_fit = model.fit()
model_fit.plot_diagnostics()

In [ ]:
#costruzione di un modello che prevede il prezzo del giorno dopo
TIMESTP = 2500 #per il train
train, test = df.Close[:TIMESTP], df.Close[TIMESTP:]
len(train), len(test)

history = [x for x in train]
y_hats = list() #nostre y
ys = list() #y vere
error_ls = list()

print('Printing Predicted vs Expected Values...')
print('\n')
for t in range(len(test)):
  model = ARIMA(history, order=(1, 1, 0)).fit()

  output = model.forecast()
  pred_value = output[0]

  original_value = test[t]
  history.append(original_value)

  #Calculating the error
  err = ((abs(pred_value - original_value)) / original_value) * 100
  error_ls.append(err)

  y_hats.append(float(pred_value))
  ys.append(float(original_value))

print('\n Mean Error in Predicting Test Case: %f ' %(sum(error_ls) / len(error_ls)), '%')

plt.figure(figsize=(8,6))
test_day= [t for t in range(len(test))]
labels = {'Original', 'Predicted'}
plt.plot(test_day, y_hats, color='green')
plt.plot(test_day, ys, color='orange')
plt.title('Expected VS Predicted Forecasting')
plt.xlabel('Day')
plt.ylabel('Closing Price')
plt.legend(labels)
plt.show()

#Image background remover

Obiettivo: costruire immagini di facce umane senza sfondo.

1. Carica un immagine di una faccia umana in posizione frontale
2. Rintraccia il rettangolo contenente la faccia (haae cascades algorith)
3. Applicare il grabCut algorith per individuare bg/fg e mask contenente l'oggetto
4. Applica un bitwise_and sull'immagine originale per ottenere l'immagine con la sola faccia e nessuno sfondo.

In [ ]:
!git clone https://github.com/ProfAI/machine-learning-avanzato.git

In [ ]:
import pathlib
import numpy as np
import cv2
import matplotlib.pylab as plt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # per printare su una stessa cella

In [ ]:
def draw_rectangle(img, start_point, end_point):
  return cv2.rectangle(img,start_point, end_point, (255, 0, 255), 2)

face_cascade = cv2.CascadeClassifier('./machine-learning-avanzato/datasets/haarcascade_frontalface_alt.xml')

In [ ]:
#Individuazione del rettangolo contenente la faccia
img = cv2.imread('./machine-learning-avanzato/datasets/face.jpg')
plt.imshow(img)

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #facciamo versione grayscale per semplificare e eliminare complessità della luminosità
gray = cv2.equalizeHist(gray) #aumentiamo contrasto
plt.imshow(gray, cmap='gray')

In [ ]:
faces = face_cascade.detectMultiScale(gray, 1.25, 15) #impostiamo threshold
len(faces) #lista di x, y, larghezza, altezza

plt.imshow(draw_rectangle(img, (faces[0][0], faces[0][1]), (faces[0][0]+faces[0][2], faces[0][1]+faces[0][3])))

In [ ]:
#ridefiniamo il tutto in maniera più analitica...
img = cv2.imread('./machine-learning-avanzato/datasets/face.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.equalizeHist(gray)
faces = face_cascade.detectMultiScale(gray, 1.25, 15)
if len(faces):
  print('Face detected...')
  for (x,y,w,h) in faces:
    _ = draw_rectangle(img, (x, y), (x+w, y+h))
_ = plt.imshow(img)



#LDA e QDA

1. calcolo del vettore delle media d-dimensionale per la classi del dataset
2. calcola le class scatter matrix
3. calcolo autovettori e autovalori della matrice di covarianza
4. ordina gli autovettori in ordine decrescente rispetto agli autovalori e sceglie i primi k autovettori con gli autovalori associati più grandi formando matrice dx k (le colonne sono autovettori)
5. usa la mtrice degli autovettoi per trasformare i vettori dello spazio originario di dimensioni 1-d di dimensioni 1-k

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # per printare su una stessa cella

In [ ]:
feature_dict = {i :label for i, label in zip(range(4), (
    'sepal length in cm',
    'sepal width in cm',
    'petal length in cm',
    'petal width in cm',
))}

In [ ]:
import pandas as pd
URL = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
df = pd.read_csv(URL, header=None)

df.columns = [el for el in sorted(feature_dict.values())] + ['class label']
df.head()

df['class label'].value_counts(dropna=False) #non ci sono valori NA

In [ ]:
#preprocessing dei dati
from sklearn.preprocessing import LabelEncoder

X = df[df.columns[:-1]].values
y = df['class label'].values

enc = LabelEncoder()
label_encoder = enc.fit(y)
y = label_encoder.transform(y) + 1

label_dict = {1: 'Setosa', 2: 'Versicolor', 3: 'Virginica'}

In [ ]:
#istogramma del dataset, si evince che le classi dividono in cluster differenziti il dataset
import matplotlib.pylab as plt
import numpy as np
import math

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12,6))
for ax, cnt in zip(axes.ravel(), range(4)):
  #set bin sizes
  min_b = math.floor(np.min(X[:cnt]))
  max_b = math.floor(np.max(X[:cnt]))
  bins = np.linspace(min_b, max_b)

  #plot histogram
  for lab, col in zip(range(4), ('blue', 'red', 'green')):
    _ = ax.hist(X[y==lab, cnt],
                color=col,
                label=f'class {label_dict[lab]}',
                bins=bins,
                alpha=0.5)

    ylims = ax.get_lim()

    #annotation
    leg = ax.legend(loc='upper right', fancybox=True, fontsize=8)
    _= leg.get_frame().set_alpha(0.5)
    _= ax.set_ylim(0, max(ylims) + 2)
    _= ax.set_xlim(feature_dict[cnt])
    _ = ax.set_title('Iris Histogram #%s ' %str(cnt + 1))

    #hide axis ticks
    _ = ax.tick_params(axis='both', which='bot', bottom='off', top='off',  labelBottom='on', left='off', right='off', labelleft='on')

    #remove axis spines
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)

  _ = axes[0][0].set_ylabel('count')
  _ = axes[1][0].set_ylabel('count')

  fig.tigh_layout()
  plt.show()

###1. calcolo dei vettori medi per ogni classe (media di ogni feature associata alla classe)

In [ ]:

np.set_printoptions(precision=4)

mean_vectors = []
for cl in range(1,4):
  mean_vectors.append(np.mean(X[y==cl], axis=0))
  print(f'Mean Vector Class {cl}: {mean_vectors[-1]}')

###2. calcolo matrice di scatter per la covarianza

In [ ]:
S_W = np.zeros((4,4)) #abbiamo 4 classi per 3 classi
for cl, mv in zip(range(1,4), mean_vectors):
  class_f = np.zeros((4,4))
  for row in X[y==cl]:
    row, mv = row.reshape(4,1), mv.reshape(4,1)
    xx = (row-mv)
    class_f += xx.dot(xx.T)
  S_W += class_f
  print(f'within-class Scatter Matrix: \n {S_W}')

overall_mean = np.mean(X, axis=0)
overall_mean = overall_mean.reshape(4,1)
S_B = np.zeros((4,4))
for i, mean_vec in enumerate(mean_vectors):
  N = len(X[y==i+1])
  mean_vec = mean_vec.reshape(4,1) #make a column vector
  xx = mean_vec - overall_mean
  S_B += N * xx.dot(xx.T)

print(f'between-class Scatter Matrix: \n {S_B}')

###3. trova autovalori e autovettori
\begin{equation}
\mathbf{S}^{-1}_w \mathbf{S}_B
\end{equation}

In [ ]:
eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))
for i in range(len(eig_vals)):
  eigvec_sc = eig_vecs[:, i].reshape(4,1)
  print('\nEigenvector {}: \n{}'.format(i+1, eigvec_sc.real))
  print('Eigenvalue {}: \n{}'.format(i+1, eig_vals[i].real))

###4. ordina gli autovettori e seleziona sottoinsieme

In [ ]:

eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,1])  for i in range(len(eig_vals))]
eig_pairs = sorted(eig_pairs, key=lambda k: k[0], reverse=True)
eig_pairs

#matrice con gli autovettori che selezioniamo
W = np.hstack((eig_pairs[0][1].reshape(4,1), eig_pairs[1][1].reshape(4,1)))
print(f'Matrix W:\n {W}')

###5. trasformiamo il dataset di partenza (vettori con dimensione 1-4) in vettori di dimensione 1,2...quindi lo proiettiamo in uno spazio a due dimensioni

In [ ]:
X_lda = X.dot(eig_pairs[0][1])
X_lda

from matplotlib.pyplot import plt

def plot_step_lda():
  ax = plt.subplot(111)
  for label, marker, color in zip(range(1,4), ('^', 's', 'o'), ('blue', 'red', 'green')):
    plt.scatter(x=X_lda[:,0].real[y==label],
                y=X_lda[:,1].real[y==label],
                marker = marker,
                color = color,
                alpha = 0.5,
                label = label_dict[label])
  plt.xlabel('LD1')
  plt.ylabel('LD2')

  leg = plt.legend(loc='upper right', fancybox=True)
  _= leg.get_frame().set_alpha(0.5)
  plt.title('LDA: Iris Projection onto the first 2 linear discriminants')

  plt.grid()
  plt.show()

plot_step_lda()

#Sistemi di raccomandazione

Esempio Content-Based

In [ ]:
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # per printare su una stessa cella

!git clone https://github.com/ProfAI/machine-learning-avanzato.git

In [ ]:
df = pd.read_csv("/content/machine-learning-avanzato/datasets/imdb_top_1000.csv")
df.head()

In [ ]:
df = df.loc[:, ["Series_Title", "Overview", "Genre"]]
df.head() #ci focalizziamo tra tre features

In [ ]:
#calcoliamo gli embedded dei contenuti, ovvero una rappresentazione vettoriale
#lavoriamo sulla overview

!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer,util

X = df['Overview'].values
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(X)
embeddings.shape

score = np.sum(np.dot(embeddings[0,:], embeddings[0,:].T) / (np.linalg.norm(embeddings[0,:,])*np.linalg.norm(embeddings[0,:,])))
score

cos_scores = util.cos_sim(embeddings, embeddings)
cos_scores.shape
cos_scores[0,1]

pairs = []
for i in range(len(cos_scores)-1):
  for j in range(i+1, len(cos_scores)):
    pairs.append({'index': (i,j), 'score': cos_scores[i,j]})

pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)

#verifichiamo sogmiglianza tra le frasi
for pair in pairs[:10]:
  i,j = pair['index']
  sent1, sent2 = X[i], X[j]
  score = pair['score']
  print(f'{sent1}|{sent2}')
  print(f'Similarity Score {score}')